In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from joblib import Parallel, delayed
from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor
import os
import warnings

warnings.filterwarnings("ignore")

# Model Selection Criteria Thresholds
p_val_thres = 0.1      # P-value threshold for individual factor significance test
correl_thres = 0.05    # Correlation threshold for single factor filtering
spss_thres = 0.01      # KPSS test threshold for stationarity test
vif_thres = 2.5        # Variance Inflation Factor threshold for multicollinearity
factor_corr_thres = 0.8  # Maximum absolute correlation between factors threshold


In [2]:
def fit_and_check(var_tuple, X, Y):
    X_feature = sm.add_constant(X[list(var_tuple)])
    model = sm.OLS(Y, X_feature).fit()
    return model


In [ ]:
def main(raw_data):
    """
    Main function to perform regression analysis with multi-factor model selection.
    
    Parameters:
    -----------
    raw_data : DataFrame
        Input data with specific structure:
        - Row 0: Group treatment codes
        - Row 1: Difference treatment flags ('Y' or 'N')
        - Row 2: Sign treatment flags ('-1' or other)
        - Row 3+: MEF factor data
        - Column 1: Y variable (logit delinquency ratio)
        - Column 2+: MEF factors
    
    Returns:
    --------
    final_result : DataFrame
        Model results with all tested combinations
    stationary_output : DataFrame
        Stationarity test results for all factors
    roll_corr : DataFrame
        Rolling correlation between each factor and Y
    """
    
    # Extract data starting row
    start_row = raw_data.iloc[:, 1].first_valid_index()
    
    # Extract features (X) and target (Y)
    X = raw_data.iloc[3:,2:].reset_index(drop=True)
    X = X.astype('float64')
    
    Y = raw_data.iloc[start_row:,1].reset_index(drop=True)
    Y = Y.astype('float64')
    
    # Extract treatment flags
    # diff_treatment = raw_data.iloc[1,2:]
    # sign_treatment = raw_data.iloc[2,2:]
    group_treatment = raw_data.iloc[0,2:]
    
    # # Apply sign treatment: multiply by -1 if sign_treatment is '-1'
    # for i in range(len(X.columns)):
    #     if sign_treatment[i] == '-1' or sign_treatment[i] == -1:
    #         X.iloc[:,i] = X.iloc[:,i] * -1
    #     else:
    #         X.iloc[:,i] = X.iloc[:,i]
    
    # Rename columns with group treatment suffix
    for i in range(len(X.columns)):
        X.rename(columns={X.columns[i]: X.columns[i]+'_'+group_treatment[i]}, inplace=True)
    
    # # Create differenced variables if diff_treatment is 'Y'
    # X_diff = X.copy()
    # X_diff.loc[:] = np.nan
    # for i in range(len(X.columns)):
    #     if diff_treatment[i] == 'Y':
    #         X_diff.iloc[:,i] = X.iloc[:,i].diff()
    
    # X_diff = X_diff.dropna(axis=1, how='all')
    # X_diff = X_diff.add_suffix('_diff')
    
    # # Combine original and differenced variables
    # X = pd.concat([X, X_diff], axis=1)
    # X = X.iloc[start_row-3:,:].reset_index(drop=True)
    
    # ============================================================================
    # SINGLE FACTOR FILTERING
    # ============================================================================
    
    # Calculate rolling correlation between each factor and Y
    # This measures the correlation strength of each individual factor with the target
    roll_corr = X.copy()
    roll_corr.loc[:] = np.nan
    
    for i in range(2, len(roll_corr)+1):
        for j in range(len(roll_corr.columns)):
            roll_corr.iloc[i-1, j] = np.corrcoef(X.iloc[:i, j], Y[:i])[0, 1]
    
    # Perform KPSS stationarity test for each factor
    # Stationary time series are required for reliable regression analysis
    stationary_df = X.copy()
    stationary_df.loc[:] = np.nan
    stationary_df = stationary_df.head(1)
    
    for i in range(len(stationary_df.columns)):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=sm.tools.sm_exceptions.InterpolationWarning)
            stationary_df.iloc[0, i] = sm.tsa.stattools.kpss(X.iloc[:, i], regression='ct')[1]
    
    # Prepare stationarity test output
    stationary_output = np.transpose(stationary_df)
    stationary_output.columns = ['Test_Value']
    stationary_output['Result'] = ['Pass' if val > spss_thres else 'Fail' 
                                    for val in stationary_output['Test_Value']]
    
    # Filter out factors that fail stationarity test or have low correlation with Y
    X = X.drop(columns=X.columns[((stationary_df < spss_thres).squeeze() | 
                                   (roll_corr.iloc[-1] < correl_thres))])
    
    # ============================================================================
    # MULTI-FACTOR MODEL FITTING
    # ============================================================================
    
    factor_corr_matrix = X.corr()
    model_list = []
    
    # Test models with 2, 3, and 4 factors
    for n in range(2, 5):
        print(f"Testing N = {n} !")
        
        # Generate all possible N-factor combinations
        var_combinations = list(combinations(pd.Index([col for col in X.columns]), n))
        
        # Filter out combinations with repeated suffixes (same group)
        var_combinations_filtered = []
        for comb in var_combinations:
            combination_suffixes = [col[-3:] for col in comb]
            has_repeated_suffixes = False
            
            for i in range(len(combination_suffixes)):
                for j in range(i + 1, len(combination_suffixes)):
                    if combination_suffixes[i] == combination_suffixes[j]:
                        has_repeated_suffixes = True
                        break
                if has_repeated_suffixes:
                    break
            
            if not has_repeated_suffixes:
                var_combinations_filtered.append(comb)
        
        var_combinations = [tuple(comb) for comb in var_combinations_filtered]
              
        # Fit models in parallel
        with Parallel(n_jobs=-1) as parallel:
            model_n_list = parallel(delayed(fit_and_check)(var_tuple, X, Y) 
                                    for var_tuple in var_combinations)
        
        model_n_list = [model for model in model_n_list if model is not None]
        model_list.extend(model_n_list)
        print(f"  - Completed: {len(model_n_list)} models fitted")
    
    # ============================================================================
    # EXTRACT MODEL RESULTS
    # ============================================================================
    
    final_result = pd.DataFrame(columns=['Factor_1', 'Factor_2', 'Factor_3', 'Factor_4', 
                                         'Adj. R-Squared', 'Intercept', 
                                         'Coefficient_1', 'Coefficient_2', 'Coefficient_3', 'Coefficient_4',
                                         'P-Value_Intercept', 'P-Value_1', 'P-Value_2', 'P-Value_3', 'P-Value_4',
                                         'VIF_1', 'VIF_2', 'VIF_3', 'VIF_4', 'Max_Factor_Correlation',
                                         'Max_Corr_Factor_1', 'Max_Corr_Factor_2'])
    
    model_info_list = []
    
    for model in model_list:
        factors = model.params.index[1:5]
        adj_r_squared = model.rsquared_adj
        intercept = model.params[0]
        coefficients = model.params[1:5]
        p_values = model.pvalues.values[:5]
        vif_factors = [variance_inflation_factor(model.model.exog, j) 
                       for j in range(1, len(factors)+1)]
        
        # Calculate maximum absolute correlation between factors
        # Extract factor names (excluding 'const')
        factor_names = [f for f in factors if f != 'const']
        max_corr = 0.0
        max_corr_factor_1 = ''
        max_corr_factor_2 = ''
        
        if len(factor_names) > 1:
            # Use pre-computed correlation matrix for fast lookup
            # Extract submatrix and find max absolute value (excluding diagonal)
            submatrix = factor_corr_matrix.loc[factor_names, factor_names].values
            # Get upper triangle (excluding diagonal) and find max absolute value
            n = len(factor_names)
            upper_triangle = np.triu(submatrix, k=1)
            abs_upper_triangle = np.abs(upper_triangle)
            max_corr = abs_upper_triangle.max()
            
            # Find the indices of the maximum correlation
            if max_corr > 0:
                max_indices = np.unravel_index(np.argmax(abs_upper_triangle), abs_upper_triangle.shape)
                max_corr_factor_1 = factor_names[max_indices[0]]
                max_corr_factor_2 = factor_names[max_indices[1]]
        
        model_info = {
            'Factor_1': factors[0] if len(factors) > 0 else '',
            'Factor_2': factors[1] if len(factors) > 1 else '',
            'Factor_3': factors[2] if len(factors) > 2 else '',
            'Factor_4': factors[3] if len(factors) > 3 else '',
            'Adj. R-Squared': adj_r_squared,
            'Intercept': intercept,
            'Coefficient_1': coefficients[0] if len(coefficients) > 0 else '',
            'Coefficient_2': coefficients[1] if len(coefficients) > 1 else '',
            'Coefficient_3': coefficients[2] if len(coefficients) > 2 else '',
            'Coefficient_4': coefficients[3] if len(coefficients) > 3 else '',
            'P-Value_Intercept': p_values[0] if len(p_values) > 0 else '',
            'P-Value_1': p_values[1] if len(p_values) > 1 else '',
            'P-Value_2': p_values[2] if len(p_values) > 2 else '',
            'P-Value_3': p_values[3] if len(p_values) > 3 else '',
            'P-Value_4': p_values[4] if len(p_values) > 4 else '',
            'VIF_1': vif_factors[0] if len(vif_factors) > 0 else '',
            'VIF_2': vif_factors[1] if len(vif_factors) > 1 else '',
            'VIF_3': vif_factors[2] if len(vif_factors) > 2 else '',
            'VIF_4': vif_factors[3] if len(vif_factors) > 3 else '',
            'Max_Factor_Correlation': max_corr,
            'Max_Corr_Factor_1': max_corr_factor_1,
            'Max_Corr_Factor_2': max_corr_factor_2
        }
        
        model_info_list.append(model_info)
    
    final_result = pd.concat([final_result, pd.DataFrame(model_info_list)])
    
    # ============================================================================
    # MODEL SELECTION CRITERIA TESTS
    # ============================================================================
    
    # Calculate number of factors in each model
    # Count non-empty coefficient fields
    final_result['num_factor'] = final_result[['Coefficient_1', 'Coefficient_2', 
                                                'Coefficient_3', 'Coefficient_4']].apply(
        lambda row: sum(1 for val in row if val != ''), axis=1)
    
    # ----------------------------------------------------------------------------
    # Test 1: P-value Test
    # Criterion: Individual factor p-value under t-test is less than 10%
    # ----------------------------------------------------------------------------
    final_result['P-Value_1'] = final_result['P-Value_1'].replace('', 999)
    final_result['P-Value_2'] = final_result['P-Value_2'].replace('', 999)
    final_result['P-Value_3'] = final_result['P-Value_3'].replace('', 999)
    final_result['P-Value_4'] = final_result['P-Value_4'].replace('', 999)
    
    conditions_pvalue = [
        (final_result['P-Value_1'] > p_val_thres) & (final_result['P-Value_1'] != 999),
        (final_result['P-Value_2'] > p_val_thres) & (final_result['P-Value_2'] != 999),
        (final_result['P-Value_3'] > p_val_thres) & (final_result['P-Value_3'] != 999),
        (final_result['P-Value_4'] > p_val_thres) & (final_result['P-Value_4'] != 999),
    ]
    
    final_result = final_result.assign(
        p_value_test=np.select(conditions_pvalue, ['Fail', 'Fail', 'Fail', 'Fail'], default='Pass'))
    
    final_result['P-Value_1'] = final_result['P-Value_1'].replace(999, '')
    final_result['P-Value_2'] = final_result['P-Value_2'].replace(999, '')
    final_result['P-Value_3'] = final_result['P-Value_3'].replace(999, '')
    final_result['P-Value_4'] = final_result['P-Value_4'].replace(999, '')
    
    # ----------------------------------------------------------------------------
    # Test 2: Coefficient Sign Test
    # Criterion: All coefficients should be non-negative (business requirement)
    # ----------------------------------------------------------------------------
    final_result['Coefficient_1'] = final_result['Coefficient_1'].replace('', 999)
    final_result['Coefficient_2'] = final_result['Coefficient_2'].replace('', 999)
    final_result['Coefficient_3'] = final_result['Coefficient_3'].replace('', 999)
    final_result['Coefficient_4'] = final_result['Coefficient_4'].replace('', 999)
    
    conditions_coe = [
        (final_result['Coefficient_1'] < 0) & (final_result['Coefficient_1'] != 999),
        (final_result['Coefficient_2'] < 0) & (final_result['Coefficient_2'] != 999),
        (final_result['Coefficient_3'] < 0) & (final_result['Coefficient_3'] != 999),
        (final_result['Coefficient_4'] < 0) & (final_result['Coefficient_4'] != 999),
    ]
    
    final_result = final_result.assign(
        Coefficient_test=np.select(conditions_coe, ['Fail', 'Fail', 'Fail', 'Fail'], default='Pass'))
    
    final_result['Coefficient_1'] = final_result['Coefficient_1'].replace(999, '')
    final_result['Coefficient_2'] = final_result['Coefficient_2'].replace(999, '')
    final_result['Coefficient_3'] = final_result['Coefficient_3'].replace(999, '')
    final_result['Coefficient_4'] = final_result['Coefficient_4'].replace(999, '')
    
    # ----------------------------------------------------------------------------
    # Test 3: VIF Test
    # Criterion: Maximum Variance Inflation Factor is less than 2.5
    # ----------------------------------------------------------------------------
    final_result['VIF_1'] = final_result['VIF_1'].replace('', 999)
    final_result['VIF_2'] = final_result['VIF_2'].replace('', 999)
    final_result['VIF_3'] = final_result['VIF_3'].replace('', 999)
    final_result['VIF_4'] = final_result['VIF_4'].replace('', 999)
    
    conditions_vif = [
        (final_result['VIF_1'] > vif_thres) & (final_result['VIF_1'] != 999),
        (final_result['VIF_2'] > vif_thres) & (final_result['VIF_2'] != 999),
        (final_result['VIF_3'] > vif_thres) & (final_result['VIF_3'] != 999),
        (final_result['VIF_4'] > vif_thres) & (final_result['VIF_4'] != 999),
    ]
    
    final_result = final_result.assign(
        VIF_test=np.select(conditions_vif, ['Fail', 'Fail', 'Fail', 'Fail'], default='Pass'))
    
    final_result['VIF_1'] = final_result['VIF_1'].replace(999, '')
    final_result['VIF_2'] = final_result['VIF_2'].replace(999, '')
    final_result['VIF_3'] = final_result['VIF_3'].replace(999, '')
    final_result['VIF_4'] = final_result['VIF_4'].replace(999, '')
    
    # ----------------------------------------------------------------------------
    # Test 4: Factor Correlation Test
    # Criterion: Maximum absolute correlation should be <= 0.8, for single factor models (correlation = 0.0) or models with low correlation, test passes
    # ----------------------------------------------------------------------------
    final_result['Max_Factor_Correlation'] = pd.to_numeric(final_result['Max_Factor_Correlation'], errors='coerce')
    final_result['Max_Factor_Correlation'] = final_result['Max_Factor_Correlation'].fillna(0.0)
    final_result['Factor_Correlation_test'] = final_result['Max_Factor_Correlation'].apply(
        lambda x: 'Pass' if x <= factor_corr_thres else 'Fail')
    
    # ----------------------------------------------------------------------------
    # Final Test Result
    # Model passes only if all four tests (p-value, coefficient, VIF, factor correlation) pass
    # ----------------------------------------------------------------------------
    final_result['test_final'] = final_result[['Coefficient_test', 'p_value_test', 'VIF_test', 'Factor_Correlation_test']].apply(
        lambda row: 'Pass' if all(val == "Pass" for val in row) else 'Fail', axis=1)
    
    # Sort by adjusted R-squared in descending order
    final_result = final_result.sort_values(by='Adj. R-Squared', ascending=False).reset_index(drop=True)
    
    return (final_result, stationary_output, roll_corr)


In [ ]:
# Configuration: Set input and output folder paths
input_folder = r'C:\Users\UV665AR\OneDrive - EY\8.Fusion\MEF model\run_input'
output_folder = r'C:\Users\UV665AR\OneDrive - EY\8.Fusion\MEF model\run_output'

# Get list of CSV files in the input folder
csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

# Create output file path
output_file = os.path.join(output_folder, 'Regression_Output_test.xlsx')

# Create Excel writer object
writer = pd.ExcelWriter(output_file, engine='openpyxl')

# Process each input CSV file
for csv_file in csv_files:
    print(f'Processing {csv_file}:')
    
    # Read input data and run analysis
    input_path = os.path.join(input_folder, csv_file)
    result, stationary_output, roll_corr = main(pd.read_csv(input_path))
    
    # Write results to Excel sheets
    roll_corr.to_excel(writer, sheet_name='Rolling_Correlation', index=True)
    stationary_output.to_excel(writer, sheet_name='Stationary_Test_Result', index=True)
    worksheet_name = os.path.splitext(csv_file)[0]
    result.to_excel(writer, sheet_name=worksheet_name, index=True)
    
    print(f'Models tested: {len(result)}')
    print(f'Models passed: {len(result[result["test_final"] == "Pass"])}')
    print(f'Models failed: {len(result[result["test_final"] == "Fail"])}')

# Save and close Excel file
writer.close()
print(f'\nResults saved to: {output_file}')


Processing input_moodys.csv:
Testing N = 2 !
  - Completed: 810 models fitted
Testing N = 3 !
  - Completed: 10270 models fitted
Testing N = 4 !
  - Completed: 93905 models fitted
